In [34]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid
from loader import load_snaps_at
from multiprocessing import Pool


In [35]:
box_N = 128
box_L = 40.0
box_dx = box_L / box_N

VOL_SCALE_FACTOR = 4
vol_grid_axis = np.linspace((-box_L + box_dx) / 2, (box_L - box_dx) / 2, num=box_N//VOL_SCALE_FACTOR)
(GX, GY, GZ) = [gi.flatten() for gi in np.meshgrid(vol_grid_axis, vol_grid_axis, vol_grid_axis, indexing='ij')]

In [41]:
Psis = load_snaps_at(f"../out_remote/2022-10-23/condensation,sigma=2.5,density=0.64,lambda=0/", [0.05, 0.50, 0.95], box_L, box_N)

loading ../out_remote/2022-10-23/condensation,sigma=2.5,density=0.64,lambda=0/snap-Psi-200-9.77.mat
loaded
loading ../out_remote/2022-10-23/condensation,sigma=2.5,density=0.64,lambda=0/snap-Psi-2000-34.94.mat
loaded
loading ../out_remote/2022-10-23/condensation,sigma=2.5,density=0.64,lambda=0/snap-Psi-3800-51.55.mat
loaded


In [100]:
Space3 = get_space_grid(box_N, box_dx)
def plot_rho_over_r(Psi3):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    radius = get_radius_grid(box_N, box_dx, densest_point)
    radius_f = radius.flatten()
    sorter = np.argsort(radius_f)
    radius_s = radius_f[sorter]
    Rho_s = Rho_f[sorter]
    counts_we, edges = np.histogram(
        radius_s, weights=Rho_s, bins=np.linspace(box_dx, box_L, 400))
    counts_uw, _ = np.histogram(radius_s, bins=np.linspace(box_dx, box_L, 400))
    y = counts_we/np.maximum(counts_uw, 1)
    x = edges
    return pgo.Scatter(x=x, y=y, mode='markers')


In [116]:
def plot_est_profile(Psi3):
    Rho = get_density(Psi3)
    Rho_f = Rho.flatten()
    # densest_point_idx = np.unravel_index(np.argmax(Rho_f), shape=Rho.shape)
    # densest_point = tuple([sp[densest_point_idx] for sp in Space3])
    # amp = np.sqrt(np.average(np.sort(Rho_f)[-8:]))
    amp = np.sqrt(1)
    print(amp)
    x = np.linspace(box_dx, box_L, 400)
    y = amp**2 * 0.998309 / (1. + 0.037653 * amp * np.square(x))**8
    return pgo.Scatter(x=x, y=y, mode='lines')

In [117]:
Psis[-1].shape

(3, 128, 128, 128)

In [118]:
fig = pgo.Figure()
fig.add_trace(plot_rho_over_r(Psis[-1]))
fig.add_trace(plot_est_profile(Psis[-1]))
fig.update_yaxes(type="log")
fig.update_xaxes(type="log")
pass
fig.write_html("2022-10-23_Rho_over_r.html")

1.0
